In [ ]:
from theano.sandbox import cuda
cuda.use('gpu0')
path = "data/sign_mix_expedition3/"
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
import cv2
import os
import time
from skimage.transform import resize
batch_size=32
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(24, activation='softmax')
        ]
p=0.8
bn_fc_layers = get_bn_da_layers(p)
hand_sign_model = Sequential(conv_layers + bn_fc_layers)
hand_sign_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
batches = get_batches(path+'train', batch_size=batch_size)
hand_sign_model.load_weights(path+'models/da_conv8_1_at80.h5')
final_labels = {}
for x,y in batches.class_indices.items():
    final_labels[y]=x



WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using Theano backend.


Found 1748 images belonging to 24 classes.


In [3]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)
preds = hand_sign_model.predict_generator(test_batches, test_batches.nb_sample)
for i, x in enumerate(preds[0]):
    print(final_labels[i], x)

Found 1 images belonging to 1 classes.
a 0.0341633
b 0.00264158
c 0.1471
d 0.0193105
e 0.0139841
f 0.0251589
g 0.00602279
h 0.00917796
i 0.18866
k 0.0019959
l 0.0267834
m 0.0131079
n 0.012184
o 0.0245394
p 0.0841798
q 0.237123
r 0.00142381
s 0.00536392
t 0.00797522
u 0.00061747
v 0.000610458
w 0.000603885
x 0.0106905
y 0.126582


In [ ]:
video_capture = cv2.VideoCapture(0)
fps = 0
start = time.time()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    fps += 1

    # Draw rectangle around face
    x = 150
    y = 50
    w = 300
    h = 350
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 3)

    # Crop + process captured frame
    hand_ini = frame[60:390, 160:440]
    cv2.imwrite(path+'imgs/imgs/img-'+str(fps)+'.jpg', hand_ini)
    #hand = np.expand_dims(resize(hand_ini, (224, 224), mode='reflect'), axis=0).transpose(0, 3, 1, 2)
    # hand = resize(hand, (224, 224), mode='reflect').transpose(2,0,1)
    # Make prediction
    #my_predict = hand_sign_model.predict(hand,
    #                              batch_size=1,
    #                              verbose=0)
    test_batches = get_batches(path+'imgs', batch_size=1, shuffle=False)
    my_predict = hand_sign_model.predict_generator(test_batches, test_batches.nb_sample)
    # Predict letter
    top_prd = np.argmax(my_predict)
    for i, x in enumerate(my_predict[0]):
        print(final_labels[i], x)
    # Only display predictions with probabilities greater than 0.5
    if np.max(my_predict) >= 0.10:

        prediction_result = final_labels[top_prd]
        preds_list = np.argsort(my_predict)[0]
        pred_2 = final_labels[preds_list[-2]]
        pred_3 = final_labels[preds_list[-3]]

        width = int(video_capture.get(3) + 0.5)
        height = int(video_capture.get(4) + 0.5)

        # Annotate image with most probable prediction
        cv2.putText(frame, text=prediction_result,org=(200,450),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=2, color=(0, 0, 0),
                    thickness=6, lineType=cv2.CV_AA)
        # Annotate image with second probable prediction (displayed on bottom right)
        cv2.putText(frame, text=pred_2,org=(275,450),
                    fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=4, color=(0, 0, 0),
                    thickness=6, lineType=cv2.CV_AA)
        # Annotate image with third probable prediction (displayed on bottom right)
        cv2.putText(frame, text=pred_3,org=(350,450),
                    fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=4, color=(0, 0, 0),
                    thickness=6, lineType=cv2.CV_AA)

    # Display the resulting frame
    cv2.namedWindow('Video', cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty('Video',cv2.WND_PROP_FULLSCREEN,cv2.cv.CV_WINDOW_FULLSCREEN)
    cv2.imshow('Video', frame)
    os.remove(path+'imgs/imgs/img-'+str(fps)+'.jpg')

    # Press 'q' to exit live loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate frames per second
end = time.time()
FPS = fps/(end-start)
print("[INFO] approx. FPS: {:.2f}".format(FPS))

# Release the capture
video_capture.release()
cv2.destroyAllWindows()

Found 1 images belonging to 1 classes.
a 0.00176572
b 0.000958934
c 0.0501553
d 0.0062199
e 0.00420469
f 0.0154681
g 0.00298542
h 0.000584209
i 0.140004
k 0.00676678
l 0.0141956
m 0.010227
n 0.00654043
o 0.0212328
p 0.0923696
q 0.0300741
r 0.00247464
s 0.00374921
t 0.0061722
u 0.00159894
v 0.0128181
w 0.0370258
x 0.00977212
y 0.522637
Found 1 images belonging to 1 classes.
a 0.00196674
b 0.00150204
c 0.13582
d 0.00698841
e 0.00524708
f 0.0102451
g 0.00656362
h 0.000947609
i 0.0829494
k 0.0143176
l 0.015175
m 0.0128145
n 0.00999903
o 0.065937
p 0.131452
q 0.0402072
r 0.00624547
s 0.00545764
t 0.00848172
u 0.00424447
v 0.0364271
w 0.0670776
x 0.0136188
y 0.316315
Found 1 images belonging to 1 classes.
a 0.00149981
b 0.000846971
c 0.053498
d 0.0141076
e 0.0077091
f 0.00919498
g 0.00290351
h 0.000308435
i 0.199076
k 0.0164754
l 0.0152736
m 0.0268512
n 0.0199373
o 0.0447538
p 0.0657091
q 0.020201
r 0.0107124
s 0.0089279
t 0.0108475
u 0.0060287
v 0.0446538
w 0.0642201
x 0.0220943
y 0.334169


a 0.00774018
b 0.000480527
c 0.251076
d 0.0018533
e 0.00436584
f 0.00107301
g 0.00875492
h 0.000941321
i 0.0326529
k 0.00208118
l 0.00673745
m 0.0327339
n 0.0289878
o 0.193928
p 0.0563043
q 0.118648
r 0.00137423
s 0.0161572
t 0.0186779
u 0.00117676
v 0.00416353
w 0.00556107
x 0.00826243
y 0.196268
Found 1 images belonging to 1 classes.
a 0.00852972
b 0.000521462
c 0.243647
d 0.00138496
e 0.00454113
f 0.00158658
g 0.00547261
h 0.00103147
i 0.0314088
k 0.00155237
l 0.00879573
m 0.023428
n 0.0188309
o 0.107684
p 0.0634782
q 0.136356
r 0.000898316
s 0.0118573
t 0.0163986
u 0.000779908
v 0.00288799
w 0.00456137
x 0.00557243
y 0.298795
Found 1 images belonging to 1 classes.
a 0.00762024
b 0.000555111
c 0.2929
d 0.00126266
e 0.00362759
f 0.000854187
g 0.00838704
h 0.00159219
i 0.011556
k 0.0010177
l 0.00595244
m 0.0286769
n 0.0237844
o 0.141452
p 0.0533924
q 0.250358
r 0.000950495
s 0.00871088
t 0.0137128
u 0.00103424
v 0.00283505
w 0.00346969
x 0.00364075
y 0.132657
Found 1 images belonging 

a 0.0332099
b 0.000410698
c 0.0674513
d 0.00202919
e 0.00568357
f 0.00244564
g 0.00425093
h 0.00102606
i 0.0379322
k 0.000890689
l 0.0156395
m 0.0214034
n 0.0220749
o 0.0221849
p 0.0157634
q 0.0634203
r 0.00033172
s 0.0104545
t 0.0353265
u 0.000344618
v 0.000887613
w 0.00192969
x 0.00350343
y 0.631405
Found 1 images belonging to 1 classes.
a 0.0140441
b 0.000476992
c 0.147395
d 0.00405888
e 0.00578449
f 0.00182719
g 0.00582393
h 0.000633805
i 0.0323286
k 0.00195494
l 0.0171219
m 0.0385496
n 0.0362022
o 0.065297
p 0.0209075
q 0.0869141
r 0.00116323
s 0.00956388
t 0.027482
u 0.00118335
v 0.00533578
w 0.00782993
x 0.00495772
y 0.463164
Found 1 images belonging to 1 classes.
a 0.015008
b 0.000523859
c 0.186174
d 0.00102069
e 0.00739859
f 0.00260692
g 0.0136375
h 0.00347947
i 0.0113225
k 0.000489318
l 0.00827751
m 0.0293992
n 0.0229172
o 0.0422712
p 0.0335703
q 0.142764
r 0.000366312
s 0.00716111
t 0.0198484
u 0.000279017
v 0.000837853
w 0.00175292
x 0.00272032
y 0.446173
Found 1 images bel

a 0.0571169
b 0.00658908
c 0.0573083
d 0.00260438
e 0.00874645
f 0.0072997
g 0.0323032
h 0.146141
i 0.0195433
k 0.0314263
l 0.0682521
m 0.0167274
n 0.0414857
o 0.0150519
p 0.195718
q 0.132624
r 0.00835579
s 0.0326602
t 0.0464735
u 0.0133725
v 0.00151765
w 0.000671204
x 0.0387601
y 0.0192508
Found 1 images belonging to 1 classes.
a 0.0110834
b 0.00165433
c 0.0476255
d 0.000822096
e 0.00445148
f 0.00225218
g 0.0451949
h 0.502623
i 0.00325504
k 0.0303711
l 0.0712116
m 0.00412833
n 0.0104705
o 0.00990117
p 0.137788
q 0.0588766
r 0.00657973
s 0.0123209
t 0.0116918
u 0.00814065
v 0.00141598
w 0.000331897
x 0.011657
y 0.00615255
Found 1 images belonging to 1 classes.
a 0.0183675
b 0.00156623
c 0.0234272
d 0.00154651
e 0.00385389
f 0.00317755
g 0.040851
h 0.360172
i 0.0116019
k 0.107742
l 0.220371
m 0.0036319
n 0.0105853
o 0.00517197
p 0.0728905
q 0.0276679
r 0.00914834
s 0.0135378
t 0.0170889
u 0.0120077
v 0.00270442
w 0.000709994
x 0.0152133
y 0.0169656
Found 1 images belonging to 1 classes.

a 0.00366851
b 0.000333942
c 0.295776
d 0.00406195
e 0.00212948
f 0.00248318
g 0.0051374
h 0.022733
i 0.00504062
k 0.00555368
l 0.569098
m 0.000959075
n 0.001817
o 0.00818235
p 0.0153026
q 0.00790907
r 0.00167955
s 0.00258516
t 0.00221415
u 0.00420235
v 0.00582124
w 0.000785464
x 0.00570218
y 0.0268236
Found 1 images belonging to 1 classes.
a 0.00447894
b 0.000283794
c 0.121492
d 0.0052025
e 0.00248112
f 0.00232041
g 0.00416439
h 0.0161215
i 0.00400625
k 0.00469123
l 0.768643
m 0.00133265
n 0.00281698
o 0.00299108
p 0.00632565
q 0.00475177
r 0.00162953
s 0.00197773
t 0.00265761
u 0.00476238
v 0.00528798
w 0.000583121
x 0.00374863
y 0.0272493
Found 1 images belonging to 1 classes.
a 0.00275136
b 0.00015362
c 0.150515
d 0.00311346
e 0.000933662
f 0.00114474
g 0.00670434
h 0.0153622
i 0.00160973
k 0.0036094
l 0.753803
m 0.000761521
n 0.00216683
o 0.00326562
p 0.00820213
q 0.00551632
r 0.00119025
s 0.000936162
t 0.00196487
u 0.00299803
v 0.00496408
w 0.000619615
x 0.00421423
y 0.0234999
Fo

...

Found 1 images belonging to 1 classes.
a 2.29234e-05
b 7.69625e-06
c 0.000278678
d 1.31569e-06
e 3.55994e-06
f 1.27404e-06
g 0.00014567
h 0.000418518
i 4.06648e-06
k 5.63647e-06
l 6.14413e-06
m 2.33603e-05
n 3.23091e-05
o 7.70679e-05
p 0.192059
q 0.806753
r 1.4552e-05
s 1.40288e-05
t 2.57834e-05
u 1.49886e-05
v 2.75245e-06
w 3.20828e-06
x 2.96314e-05
y 5.53568e-05
Found 1 images belonging to 1 classes.
a 3.98182e-05
b 2.13083e-05
c 0.000607636
d 3.11458e-06
e 8.90202e-06
f 3.69506e-06
g 0.000182005
h 0.00050708
i 9.25029e-06
k 1.16927e-05
l 1.1409e-05
m 5.01704e-05
n 6.50897e-05
o 0.00014232
p 0.208576
q 0.789448
r 3.38578e-05
s 3.00222e-05
t 4.81228e-05
u 3.53767e-05
v 6.13228e-06
w 6.90549e-06
x 5.9928e-05
y 9.19988e-05
Found 1 images belonging to 1 classes.
a 8.07235e-06
b 2.06631e-06
c 8.51582e-05
d 2.93791e-07
e 8.68179e-07
f 3.59278e-07
g 6.0098e-05
h 0.000339504
i 5.95422e-07
k 7.32795e-07
l 1.93064e-06
m 6.5855e-06
n 9.53484e-06
o 2.23343e-05
p 0.11796
q 0.881463
r 2.25906e-06


a 5.11861e-06
b 1.42492e-06
c 3.66143e-05
d 1.06345e-07
e 5.48207e-07
f 2.41696e-07
g 1.64858e-05
h 9.08756e-05
i 7.80092e-07
k 3.66672e-07
l 6.25375e-07
m 5.32306e-06
n 5.27854e-06
o 1.33279e-05
p 0.0945545
q 0.905239
r 9.15215e-07
s 2.48294e-06
t 4.81495e-06
u 9.48607e-07
v 1.91511e-07
w 4.75448e-07
x 2.54327e-06
y 1.74038e-05
Found 1 images belonging to 1 classes.
a 2.21744e-06
b 6.52293e-07
c 1.9763e-05
d 4.54124e-08
e 2.59607e-07
f 1.03834e-07
g 7.53326e-06
h 3.40876e-05
i 3.63704e-07
k 1.96641e-07
l 3.20432e-07
m 2.95159e-06
n 2.78945e-06
o 7.18633e-06
p 0.105239
q 0.894666
r 4.5895e-07
s 1.12275e-06
t 2.3498e-06
u 4.26242e-07
v 1.06982e-07
w 3.03789e-07
x 1.23764e-06
y 1.0659e-05
Found 1 images belonging to 1 classes.
a 5.17073e-06
b 2.02149e-06
c 3.50428e-05
d 1.38476e-07
e 8.31671e-07
f 4.57955e-07
g 2.79195e-05
h 0.000133734
i 1.23634e-06
k 7.83655e-07
l 8.45707e-07
m 6.63682e-06
n 7.24267e-06
o 1.5644e-05
p 0.308233
q 0.691481
r 1.55056e-06
s 3.90168e-06
t 7.19379e-06
u 1.32

Found 1 images belonging to 1 classes.
a 0.00031751
b 0.00103931
c 0.388959
d 0.0114934
e 0.00102124
f 0.00151727
g 0.0822826
h 0.014976
i 0.00102714
k 0.0594185
l 0.0125319
m 0.000678989
n 0.00297359
o 0.0280337
p 0.0535436
q 0.00750796
r 0.116345
s 0.00112147
t 0.00153866
u 0.039946
v 0.0367855
w 0.00475774
x 0.13068
y 0.00150313
Found 1 images belonging to 1 classes.
a 0.000309005
b 0.000156737
c 0.501381
d 0.000514107
e 0.000290013
f 0.000163711
g 0.215533
h 0.123587
i 0.00012566
k 0.00549173
l 0.0123282
m 0.000210667
n 0.000739907
o 0.0140955
p 0.0644063
q 0.0383353
r 0.00510544
s 0.000255316
t 0.000471623
u 0.00234584
v 0.00348979
w 0.000539517
x 0.00765625
y 0.00246871
Found 1 images belonging to 1 classes.
a 0.000258265
b 0.000340714
c 0.328665
d 0.000642383
e 0.000240076
f 0.00057709
g 0.0722549
h 0.0145855
i 0.000541346
k 0.033279
l 0.0311886
m 0.000257551
n 0.000690563
o 0.0127347
p 0.321573
q 0.0335963
r 0.0078407
s 0.000225406
t 0.000706935
u 0.00515876
v 0.0641999
w 0.029

a 0.00154328
b 0.000267062
c 0.119355
d 0.000307671
e 0.000206162
f 0.000226053
g 0.00589308
h 0.00418413
i 0.000210075
k 0.00123851
l 0.0531412
m 0.000848487
n 0.00146956
o 0.00407313
p 0.148137
q 0.635065
r 0.000406792
s 9.88879e-05
t 0.000985143
u 0.000722388
v 0.00185997
w 0.00101082
x 0.00039201
y 0.0183582
Found 1 images belonging to 1 classes.
a 0.00128472
b 0.00196608
c 0.633069
d 0.00179027
e 0.00161514
f 0.00490595
g 0.00965044
h 0.00437788
i 0.0128398
k 0.0326402
l 0.0369721
m 0.00139095
n 0.00194575
o 0.0450036
p 0.10234
q 0.019787
r 0.0117702
s 0.0026074
t 0.00211265
u 0.00773724
v 0.0181206
w 0.00686956
x 0.0147417
y 0.0244612
Found 1 images belonging to 1 classes.
a 0.00248885
b 0.00131776
c 0.708607
d 0.00229536
e 0.00343586
f 0.00366567
g 0.00792914
h 0.00604621
i 0.00880961
k 0.0122373
l 0.0334108
m 0.00183841
n 0.00268656
o 0.0529513
p 0.0688795
q 0.0203923
r 0.00648365
s 0.00411517
t 0.00352017
u 0.00422809
v 0.00684383
w 0.00230908
x 0.0145342
y 0.0209741
Found 1 i

a 0.000126828
b 7.89706e-05
c 0.050854
d 4.83851e-05
e 7.96824e-05
f 6.44348e-05
g 0.0138233
h 0.0208468
i 8.39719e-05
k 0.00306864
l 0.00444059
m 7.59933e-05
n 0.000180781
o 0.00313791
p 0.802258
q 0.0914233
r 0.000943918
s 0.000156472
t 0.000218831
u 0.000753274
v 0.00178536
w 0.000674269
x 0.00230657
y 0.00256966
Found 1 images belonging to 1 classes.
a 0.00101624
b 0.00398028
c 0.0976022
d 0.00220731
e 0.000598972
f 0.00123517
g 0.0764253
h 0.139991
i 0.000372654
k 0.0961211
l 0.063549
m 0.000980176
n 0.00382648
o 0.00832964
p 0.225232
q 0.0607943
r 0.0627929
s 0.000787404
t 0.00180581
u 0.0983128
v 0.0297912
w 0.00452318
x 0.0186276
y 0.00109639
Found 1 images belonging to 1 classes.
a 0.00139331
b 0.00348346
c 0.109771
d 0.00485278
e 0.00060486
f 0.00098672
g 0.0135728
h 0.0345437
i 0.000546651
k 0.135388
l 0.246711
m 0.000389401
n 0.00148155
o 0.00386601
p 0.0634834
q 0.0267348
r 0.0946732
s 0.000645697
t 0.00125245
u 0.203202
v 0.0383377
w 0.00263887
x 0.0104135
y 0.00102768
Fo

In [2]:
import os

In [14]:
test_batches = get_batches(path+'imgs', batch_size=batch_size, shuffle=False)

Found 1 images belonging to 1 classes.


In [13]:
path

'data/sign_mix_expedition2/'

In [2]:
hand_sign_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[1][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[1][0]            
___________________________________________________________________________________________

dense_6 (Dense)                  (None, 24)            6168        dropout_5[0][0]                  
Total params: 21,211,480
Trainable params: 21,210,456
Non-trainable params: 1,024
____________________________________________________________________________________________________
